# Rasterize: Vector(shp) or Coordinates to Image(GeoTiff)

## General Import

In [2]:
from tqdm.auto import tqdm
from osgeo import gdal, ogr
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyproj import Proj, Transformer
import rasterio
from rasterio.mask import mask
from shapely.geometry import box

import os

def mkdir(path):
    os.makedirs(path)
def is_exist_dir(path):
    if not os.path.exists(path):
        mkdir(path)

# [Convert Function]

### Vector to Raster, SHP to TIFF
- Input: SHP 파일
- Output: Tiff 파일
- Reference: Tiff 파일

- Reference의 정보(좌표계, 해상도) + Input 데이터의 Geometry정보를 이용해 새로운 Tiff 생성
    - 

In [ ]:
from osgeo import ogr, gdal
import subprocess
import shutil

def rasterize_vector(InputVector, OutputImage, RefImage, value):
    gdalformat = 'GTiff'
    datatype = gdal.GDT_Float32
    burnVal = 1

    ##########################################################
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    # Open Shapefile
    Shapefile = ogr.Open(InputVector)
    Shapefile_layer = Shapefile.GetLayer()
    
    Feature = Shapefile_layer.GetNextFeature()
    Geometry = Feature.GetGeometryRef()
    Value = Feature.GetField(value)

    # Rasterise
    print("Rasterising shapefile...")
    open(OutputImage,'w+').close()  # create outputfile if not exists
    Output = gdal.GetDriverByName(gdalformat).Create(OutputImage, Image.RasterXSize, Image.RasterYSize, 1, datatype, options=['COMPRESS=DEFLATE'])
    Output.SetProjection(Image.GetProjectionRef())
    Output.SetGeoTransform(Image.GetGeoTransform()) 

    # Write data to band 1
    Band = Output.GetRasterBand(1)
    Band.SetNoDataValue(-999)  # InputVector에서 지정되지 않은 영역에 대해 채울 값 지정
    gdal.RasterizeLayer(Output, [1], Shapefile_layer, options = ["ATTRIBUTE=" + value])

    # Close datasets
    Band = None
    Output = None
    Image = None
    Shapefile = None

    # Build image overviews
    subprocess.call("gdaladdo --config COMPRESS_OVERVIEW DEFLATE "+OutputImage+" 2 4 8 16 32 64", shell=True)
    print("Done.")

In [47]:
from pyidw import idw # 보간법 실행

idw.idw_interpolation(
    input_point_shapefile=f'220304_renew.shp', # 보간하고자 하는 shp 파일 
    extent_shapefile="boundary/boundary.shp", # 경계 shp 파일(현재 강원도)
    column_name='z', # 보간하고자 하는 feature 이름. 
    power=2, # 거리 가중치 계수 
    search_radious=8, # 검색하고자 하는 범위 
    output_resolution=400, # 결과물 해상도 
)

### Raster 간 해상도 통일 & 해상도 통일 후 테두리 정리

In [6]:
from osgeo import gdal, ogr
import subprocess


def match_resolution(InputImage, OutputImage, RefImage):
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    output_width = Image.RasterXSize
    output_height = Image.RasterYSize

    # Construct the command string
    open(OutputImage, 'w')
    command = ['gdal_translate', '-outsize', str(output_width), str(output_height), '-r', 'bilinear', InputImage, OutputImage]

    # Run the command
    subprocess.run(command)

    # Set geotransform and projection information
    Output = gdal.Open(OutputImage)
    Output.SetGeoTransform(Image.GetGeoTransform())
    Output.SetProjection(Image.GetProjection())

    Image = None
    Output = None
        
def clear_boundary_using_arr(InputImage, OutputImage, RefImage):
    InputArr = image_to_array(InputImage)
    Image = gdal.Open(RefImage, gdal.GA_Update)
    ImageArr = Image.ReadAsArray()
    
    out_boundary_lat = []
    out_boundary_lon = []
    out_value = ImageArr[0, 0]
    for i in range(Image.RasterYSize):
        for j in range(Image.RasterXSize):
            if ImageArr[i, j] == out_value:
                out_boundary_lat.append(i)
                out_boundary_lon.append(j)
    InputArr[out_boundary_lat, out_boundary_lon] = InputArr[0, 0]
    array_to_image(InputArr, OutputImage, RefImage)
    


In [17]:
InputImage='../data/geo_data/raw/population_density_gw.tif'
OutputImage='../data/geo_data/raw/population_density_gw2.tif'
RefImage='../data/geo_data/raw/Slope_gw.tif'
match_resolution(InputImage, OutputImage, RefImage)

In [ ]:
InputImage='Height_Final.tif'
OutputImage='height_gw.tif'
RefImage='boundary/boundary_blank_resized.tif'
clear_boundary_using_arr(InputImage, OutputImage, RefImage)

### Crop Image

In [5]:


def crop_image_using_coordinates(InputImage, OutputImage, RefImage, latitude, longitude, InputCrs, OutputCrs, CropSize):
    """extract tiff height and width"""
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)
    width = Image.RasterXSize
    height = Image.RasterYSize
    Image = None

    """extract vertex coordinates"""
    rds = rasterio.open(RefImage)
    rds.bounds
    left = rds.bounds[0]
    right = rds.bounds[2]
    top = rds.bounds[3]
    bottom = rds.bounds[1]

    """Convert coordinates system"""
    resolution_x = (right - left) / width
    resolution_y = (top - bottom) / height

    InputCrs = _STATIC_formatting_crs(InputCrs)
    OutputCrs = _STATIC_formatting_crs(OutputCrs)
    transformer = Transformer.from_crs(InputCrs, OutputCrs)
    longitude, latitude = transformer.transform(longitude, latitude)
    
    left_box = latitude - (resolution_x * CropSize)
    top_box = longitude + (resolution_y * CropSize)
    right_box = latitude + (resolution_x * CropSize)
    bottom_box = longitude - (resolution_y * CropSize)
    window = (left_box, top_box, right_box, bottom_box)

    gdal.Translate(OutputImage, InputImage, projWin = window)
    
    
InputImage = 'forestmap.tif'
OutputImage = 'test.tif'
RefImage = 'gw_boundary.tif'
latitude = 128.1385
longitude = 37.2598
InputCrs = '4326'
OutputCrs = '4326'
CropSize = 10
crop_image_using_coordinates(InputImage, OutputImage, RefImage, latitude, longitude, InputCrs, OutputCrs, CropSize)

### raster to np.array & np.array to raster

In [5]:
def image_to_array(InputImage):
    Image = gdal.Open(InputImage, gdal.GA_Update)
    array = Image.ReadAsArray()
    print(array.shape)
    return array

def array_to_image(InputArr, OutputImage, RefImage):
    Image = gdal.Open(RefImage, gdal.GA_Update)
    ImageArr = Image.ReadAsArray()
    
    open(OutputImage, 'w')
    Output = gdal.GetDriverByName('GTiff').Create(OutputImage, ImageArr.shape[1], ImageArr.shape[0], 1, gdal.GDT_Float32)
    #writting output raster
    Output.GetRasterBand(1).WriteArray(InputArr)
    Output.SetProjection(Image.GetProjection())
    Output.SetGeoTransform(Image.GetGeoTransform())

    Image = None
    Output = None
    
InputArr=np.load('array.npy')
OutputImage='result_1.tif'
RefImage='boundary_blank_resized.tif'
array_to_image(InputArr, OutputImage, RefImage)

InputImage = 'data/reference/gw_boundary.tif'
arr = image_to_array(InputImage)

## crop gangwon boundary 

In [15]:
shapefile = gpd.read_file("../data/gw_boundary/boundary.shp")

path = '../data/geo_data/raw/origin/Landuse.tif'

# Read the raster dataset
with rasterio.open(path) as src:
    if shapefile.crs != src.crs:
        shapefile = shapefile.to_crs(src.crs)

    raster_bounds = box(*src.bounds)
    shapefile = shapefile[shapefile.geometry.intersects(raster_bounds)]
    clipped_raster, transform = mask(src, shapefile.geometry, crop=True)

    with rasterio.open(
        "../data/geo_data/raw/Landuse_gw.tif",
        "w",
        driver="GTiff",
        height=clipped_raster.shape[1],
        width=clipped_raster.shape[2],
        count=clipped_raster.shape[0],
        dtype=clipped_raster.dtype,
        crs=src.crs,
        transform=transform,
    ) as dst:
        # Write the clipped subset to the output file
        dst.write(clipped_raster, indexes=list(range(1, clipped_raster.shape[0] + 1)))

print("완료")

완료
